In [1]:
# coding=utf-8
# Copyright 2021 DAF Trucks NV.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lint as: python3
"""Trains TFT based on a defined set of parameters.

Uses default parameters supplied from the configs file to train a TFT model from
scratch.

Usage:
python3 script_train_fixed_params {expt_name} {output_folder}

Command line args:
  expt_name: Name of dataset/experiment to train.
  output_folder: Root folder in which experiment is saved


"""

import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model_2
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow as tf
import data_formatters.daf_extended

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model_2.TemporalFusionTransformer
DafFormatter = data_formatters.daf_extended.DafExtendedFormatter()


def main(expt_name,
         model_folder,
         data_csv_path,
         data_formatter,
         use_testing_mode=False):
  """Trains tft based on defined model params.

  Args:
    expt_name: Name of experiment
    model_folder: Folder path where models are serialized
    data_csv_path: Path to csv file containing data
    data_formatter: Dataset-specific data fromatter (see
      expt_settings.dataformatter.GenericDataFormatter)
    use_testing_mode: Uses a smaller models and data sizes for testing purposes
      only -- switch to False to use original default settings
  """

  num_repeats = 1

  if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

  # Tensorflow setup
  # default_keras_session = tf.keras.backend.get_session()

  # if use_gpu:
  #   tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

  # else:
  #   tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

  print("*** Training from defined parameters for {} ***".format(expt_name))

  print("Loading & splitting data...")
  raw_data = pd.read_csv(data_csv_path, index_col=0)
  train, valid, test = data_formatter.split_data(raw_data)
  train_samples, valid_samples = data_formatter.get_num_samples_for_calibration(
  )

  # Sets up default params
  fixed_params = data_formatter.get_experiment_params()
  params = data_formatter.get_default_model_params()
  params["model_folder"] = model_folder

  # Parameter overrides for testing only! Small sizes used to speed up script.
  if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 5
    train_samples, valid_samples = 100, 10

  # Sets up hyperparam manager
  print("*** Loading hyperparm manager ***")
  opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                     fixed_params, model_folder)

  # Training -- one iteration only
  print("*** Running calibration ***")
  print("Params Selected:")
  for k in params:
    print("{}: {}".format(k, params[k]))

  best_loss = np.Inf
  for _ in range(num_repeats):

    # tf.reset_default_graph()
    # with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

      # tf.keras.backend.set_session(sess)

      params = opt_manager.get_next_parameters()
      model = ModelClass(params)

      if not model.training_data_cached():
        model.cache_batched_data(train, "train", num_samples=train_samples)
        model.cache_batched_data(valid, "valid", num_samples=valid_samples)

      # sess.run(tf.global_variables_initializer())
      model.fit()

      val_loss = model.evaluate()

      if val_loss < best_loss:
        opt_manager.update_score(params, val_loss, model)
        best_loss = val_loss

      # tf.keras.backend.set_session(default_keras_session)

  print("*** Running tests ***")
  # tf.reset_default_graph()
  # with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    # tf.keras.backend.set_session(sess)
  best_params = opt_manager.get_best_params()
  model = ModelClass(best_params)

  model.load(opt_manager.hyperparam_folder)

  print("Computing best validation loss")
  val_loss = model.evaluate(valid)

  print("Computing test loss")
  output_map = model.predict(test, return_targets=True)
  targets = data_formatter.format_predictions(output_map["targets"])
  p50_forecast = data_formatter.format_predictions(output_map["p50"])
  p90_forecast = data_formatter.format_predictions(output_map["p90"])

  def extract_numerical_data(data):
    """Strips out forecast time and identifier columns."""
    return data[[
        col for col in data.columns
        if col not in {"forecast_time", "identifier"}
    ]]

  p50_loss = utils.numpy_normalised_quantile_loss(
      extract_numerical_data(targets), extract_numerical_data(p50_forecast),
      0.5)
  p90_loss = utils.numpy_normalised_quantile_loss(
      extract_numerical_data(targets), extract_numerical_data(p90_forecast),
      0.9)

    # tf.keras.backend.set_session(default_keras_session)

  print("Training completed @ {}".format(dte.datetime.now()))
  print("Best validation loss = {}".format(val_loss))
  print("Params:")

  for k in best_params:
    print(k, " = ", best_params[k])
  print()
  print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
      p50_loss.mean(), p90_loss.mean()))

  return output_map


model = main(
      expt_name='daf',
      model_folder='tft_outputs/saved_models/daf/fixed',
      data_csv_path='tft_outputs/data/daf/tf_input.csv',
      data_formatter=DafFormatter,
      use_testing_mode=False)  # Change to false to use original default params



2.5.0
*** Training from defined parameters for daf ***
Loading & splitting data...
Processed 0 out of 9461 drivers
Processed 1000 out of 9461 drivers
Processed 2000 out of 9461 drivers
Processed 3000 out of 9461 drivers
Processed 4000 out of 9461 drivers
Processed 5000 out of 9461 drivers
Processed 6000 out of 9461 drivers
Processed 7000 out of 9461 drivers
Processed 8000 out of 9461 drivers
Processed 9000 out of 9461 drivers
Setting scalers with training data...
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.2
hidden_layer_size: 20
learning_rate: 0.001
minibatch_size: 64
max_gradient_norm: 1.0
num_heads: 4
stack_size: 1
model_folder: tft_outputs/saved_models/daf/fixed
Resetting temp folder
Cached data "train" updated
Cached data "valid" updated
*** Fitting TemporalFusionTransformer ***
Getting batched_data
Using cached training data
Using cached validation data
Using keras standard fit
Epoch 1/100
21418/21418 [===========================

In [2]:
targets = list(model['targets']['t+0'].values)
p50 = list(model['p50']['t+0'].values)
p90 = list(model['p90']['t+0'].values)
p10 = list(model['p10']['t+0'].values)

In [3]:
target_unscaled = DafFormatter._target_scaler.inverse_transform(targets)
p50_unscaled = DafFormatter._target_scaler.inverse_transform(p50)
p90_unscaled = DafFormatter._target_scaler.inverse_transform(p90)
p10_unscaled = DafFormatter._target_scaler.inverse_transform(p10)

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('RMSE p10: {}'.format(mean_squared_error(targets, p10, squared=False)))
print('MAE p10: {}'.format(mean_absolute_error(targets, p10)))
print('RMSE p50: {}'.format(mean_squared_error(targets, p50, squared=False)))
print('MAE p50: {}'.format(mean_absolute_error(targets, p50)))
print('RMSE p90: {}'.format(mean_squared_error(targets, p90, squared=False)))
print('MAE p90: {}'.format(mean_absolute_error(targets, p90)))

RMSE p10: 0.8062184305164593
MAE p10: 0.657649949352054
RMSE p50: 0.49624124053611524
MAE p50: 0.37209188725085757
RMSE p90: 0.7670339460859574
MAE p90: 0.6235325115261202


In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('RMSE p10 unscaled: {}'.format(mean_squared_error(target_unscaled, p10_unscaled, squared=False)))
print('MAE p10 unscaled: {}'.format(mean_absolute_error(target_unscaled, p10_unscaled)))
print('RMSE p50 unscaled: {}'.format(mean_squared_error(target_unscaled, p50_unscaled, squared=False)))
print('MAE p50 unscaled: {}'.format(mean_absolute_error(target_unscaled, p50_unscaled)))
print('RMSE p90 unscaled: {}'.format(mean_squared_error(target_unscaled, p90_unscaled, squared=False)))
print('MAE p90 unscaled: {}'.format(mean_absolute_error(target_unscaled, p90_unscaled)))

RMSE p10 unscaled: 14.866357462827747
MAE p10 unscaled: 12.126811826560397
RMSE p50 unscaled: 9.150497570464788
MAE p50 unscaled: 6.861231263014301
RMSE p90 unscaled: 14.143811155347052
MAE p90 unscaled: 11.49769996280489
